In [ ]:
# этот скрипт делает запрос к openai api.

In [ ]:
#%pip install numpy
#%pip install pandas
#%pip install python-dotenv

In [ ]:
import base64
import requests
from dotenv import load_dotenv
import os
import json

# Загрузка переменных из файла .env
load_dotenv()

# Получение значения переменной OPENAI_API_KEY
api_key = os.getenv('OPENAI_API_KEY')

headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [ ]:
# Path to your image
image_path = "img/1T.png"

# Getting the base64 string
base64_image = encode_image(image_path)

In [ ]:
payload = {
  "model": "gpt-4o",
  "messages": [
    {
      "role": "system", 
      "content": "You are a professional quantitative analyst specializing in financial market analysis and price" 
              + " forecasting. You always spend a few sentences explaining background context, assumptions, and" 
              + " step-by-step thinking BEFORE you try to answer a question to ensure accuracy. You conduct a"
              + " thorough analysis of the overall market situation before providing forecasts."
    },
    {
      "role": "user", 
      "content": [
        {
          "type": "text", 
          "text": "Could you please analyze the attached minute candlestick chart and provide forecasts for the next"
                + " 5 minutes. The main chart includes Bollinger Bands (period=20, color=black), VWAP (color=blue), ATR"
                + " (period=10, color=red). The volume chart also includes RSI (period=14, color=black)."
                + " Below the volume chart is a panel with MACD (fast=12, slow=26, signal=9). Please MAKE SURE TO"
                + " INCLUDE the predicted CLOSING PRICE RANGE after the next 5 minutes as well as the PROBABILITIES of"
                + " price increase and decrease."
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}"
          },
        }
      ],
    }
  ],
  "max_tokens": 1000
}

# делаем первый запрос к API, в котором просим провести анализ графика.
response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
j = response.json()
j

In [ ]:
# эта ячейка берет ответ из предыдущей ячейки и вставляет его в маркдаун ячейку ниже, чтобы было удобно читать.

# Запустите следующий код в Jupyter Notebook, чтобы создать функцию, которая будет добавлять новую ячейку с содержимым:
from IPython.display import display, Markdown, Javascript

def create_markdown_cell(content):
    display(Markdown(content))

# Создайте функцию, которая удаляет следующую ячейку, если она имеет тип Markdown:
def delete_next_markdown_cell():
    display(Javascript("""
    var currentCell = Jupyter.notebook.get_selected_index();
    var nextCell = Jupyter.notebook.get_cell(currentCell + 1);
    if (nextCell && nextCell.cell_type === 'markdown') {
        Jupyter.notebook.delete_cell(currentCell + 1);
    }
    """))

# Извлечение значения поля content
content = j['choices'][0]['message']['content']

# Удаление следующей Markdown ячейки, если она существует
delete_next_markdown_cell()

# Создание новой markdown ячейки с содержимым
create_markdown_cell(content)

In [ ]:
content

In [ ]:
# в этой ячейке делаем второй запрос к API, в котором просим предыдущий ответ оформить в виде json.

payload = {
  "model": "gpt-3.5-turbo",# берем модель 3.5 турбо, т.к. она в 10 раз дешевле gpt-4o и с задачей справляется.
  "response_format": { "type": "json_object" },
  "messages": [
    {
      "role": "system", 
      "content": """You are an LLM assistant specialized in converting financial analysis text into structured JSON format.
 Your task is to extract key numerical values and probabilities from the provided analysis and format
 them into a JSON object with the following fields: 'h' for the predicted highest price, 'l' for the
 predicted lowest price, 'p_up' for the probability of price increase, 'p_down' for the probability
 of price decrease, and 'error' for any potential issues or discrepancies noted during extraction.
 Ensure the JSON output accurately reflects the information in the analysis, adheres to the following
 JSON Schema:

```
{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "type": "object",
  "properties": {
    "h": {
      "type": "number",
      "description": "Predicted highest price"
    },
    "l": {
      "type": "number",
      "description": "Predicted lowest price"
    },
    "p_up": {
      "type": "number",
      "description": "Probability of price increase",
      "minimum": 0,
      "maximum": 100
    },
    "p_down": {
      "type": "number",
      "description": "Probability of price decrease",
      "minimum": 0,
      "maximum": 100
    },
    "error": {
      "type": "string",
      "description": "Error message if any issue is found"
    }
  },
  "additionalProperties": false
}
```

Respond only in JSON format without any additional text, as your response will be processed automatically by a program."""
    },
    {
      "role": "user", 
      "content": [
        {
          "type": "text", 
          "text": content
        }
      ],
    }
  ],
  "max_tokens": 1000
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
j = response.json()
j

In [ ]:
json_string = j['choices'][0]['message']['content']

# Преобразование строки JSON в объект Python (словарь)
data = json.loads(json_string)

# Чтение значения 'h'
data["h"], data['l'], data['p_up'], data['p_down'], data['error']